# Reto: Análisis de Ventas de una Tienda en Línea
## Descripción del Desafío:

- Tienes que conectar a una base de datos Cassandra.
- Crear una tabla que registre las ventas de una tienda en línea.
- Insertar datos en la tabla a partir de consultas predefinidas.
- Realizar dos consultas sobre los datos insertados.

### Tabla de Ventas:
La tabla se llamará sales.

Las columnas serán:
- order_id (UUID): Identificador único de la orden.
- customer_id (UUID): Identificador único del cliente.
- product_id (UUID): Identificador único del producto.
- quantity (int): Cantidad del producto vendido.
- order_date (timestamp): Fecha y hora de la orden.
- total_amount (decimal): Monto total de la orden.

Consultas Requeridas:

> - Consulta 1: Obtener el total de ventas realizadas en un rango de fechas específico.
> - Consulta 2: Obtener el total de productos vendidos por cada cliente.

#### Paso 1: Conectar a Cassandra
Primero, necesitas instalar el paquete cassandra-driver si no lo tienes instalado:

In [1]:
#!pip install cassandra-driver

In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import SimpleStatement
from datetime import datetime, timedelta
import uuid, random, decimal

## Conexión a Cassandra

Primero, conectamos a nuestro clúster de Cassandra. Asegúrate de que el clúster está corriendo y accesible desde tu máquina.

In [2]:
# Conexión a Cassandra
# cloud_config= {
#        'secure_connect_bundle': '<path_to_secure_connect_bundle>'
# }
# auth_provider = PlainTextAuthProvider('<client_id>', '<client_secret>')
# cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
# session = cluster.connect('<keyspace_name>')

In [3]:
# Conectar a Cassandra
cluster = Cluster(['127.0.0.1'])  # Sustituye con la dirección de tu clúster
session = cluster.connect()

#### Paso 2: Script de Conexión y Creación de Tabla
Aquí tienes un script en Python que puedes usar para conectar a Cassandra y crear la tabla:

## Crear un Keyspace

Un keyspace en Cassandra es un contenedor para las tablas. Aquí creamos un keyspace llamado `sales_data`.

In [4]:
# Crear un keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sales_data
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}
""")

## Seleccionar el Keyspace

Después de crear el keyspace, lo seleccionamos para que las siguientes operaciones se apliquen dentro de este contexto.

In [5]:
# Seleccionar el keyspace
session.set_keyspace('sales_data')

#### Paso 3: Creación de Tabla
Aquí tienes un script en Python que puedes usar para crear la tabla:

In [6]:
# Creación de la tabla de ventas
create_table_query = """
CREATE TABLE IF NOT EXISTS sales (
    order_id UUID,
    customer_id UUID,
    product_id UUID,
    quantity INT,
    order_date TIMESTAMP,
    total_amount DECIMAL,
    PRIMARY KEY (order_id, customer_id)
)  WITH CLUSTERING ORDER BY (customer_id DESC);
"""
session.execute(create_table_query)

print("Tabla 'sales' creada exitosamente.")


Tabla 'sales' creada exitosamente.


#### Paso 4: Insertar Datos en la Tabla
Ahora, insertemos algunos datos de ejemplo en la tabla:

In [7]:
insert_data_query = """
INSERT INTO sales (order_id, customer_id, product_id, quantity, order_date, total_amount)
VALUES (%s, %s, %s, %s, %s, %s)
"""

order_id = uuid.uuid4()
customer_id = uuid.uuid4()
product_id = uuid.uuid4()
quantity = 3
order_date = datetime.now()
total_amount = 150.75

session.execute(insert_data_query, (order_id, customer_id, product_id, quantity, order_date, total_amount))

print("Datos insertados exitosamente.")


Datos insertados exitosamente.


#### Paso 5: Realizar Consultas
Finalmente, aquí tienes las consultas requeridas:

In [8]:
# Consulta 1: Total de ventas en un rango de fechas
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

query1 = """
SELECT SUM(total_amount) AS total_sales
FROM sales
WHERE order_date >= %s AND order_date <= %s ALLOW FILTERING
"""
result1 = session.execute(query1, (start_date, end_date))
for row in result1:
    print(f"Total de ventas: {row.total_sales}")

Total de ventas: 150.75


In [9]:
# Consulta 2: Total de productos vendidos por cliente
query2 = """
SELECT order_id, SUM(quantity) AS total_products
FROM sales
GROUP BY order_id
"""
result2 = session.execute(query2)
for row in result2:
    print(f"Cliente {row.order_id} ha comprado {row.total_products} productos")

Cliente 6cc3c07f-a626-4270-a357-ece2096dd257 ha comprado 3 productos


## Insertar Datos de Ejemplo

Insertamos datos de ejemplo en la tabla `sales`.

In [10]:
def insert_artificial_data(session, num_records, start_date, end_date, use_ttl=False, ttl_seconds=0):
    insert_data_query_with_ttl = """
    INSERT INTO sales (order_id, customer_id, product_id, quantity, order_date, total_amount)
    VALUES (%s, %s, %s, %s, %s, %s) USING TTL %s
    """
    
    insert_data_query_without_ttl = """
    INSERT INTO sales (order_id, customer_id, product_id, quantity, order_date, total_amount)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    delta = end_date - start_date
    
    for _ in range(num_records):
        order_id = uuid.uuid4()
        customer_id = uuid.uuid4()
        product_id = uuid.uuid4()
        quantity = random.randint(1, 10)
        order_date = start_date + timedelta(days=random.randint(0, delta.days),seconds=random.randint(0, 86400))
        total_amount = decimal.Decimal(random.uniform(10.0, 500.0)).quantize(decimal.Decimal('0.01'))
        
        if use_ttl:
            session.execute(insert_data_query_with_ttl, (order_id, customer_id, product_id, quantity, order_date, total_amount, ttl_seconds))
        else:
            session.execute(insert_data_query_without_ttl, (order_id, customer_id, product_id, quantity, order_date, total_amount))
    
    print(f"Se han insertado {num_records} registros artificiales en la tabla 'sales'.")

In [11]:
# Parámetros
num_records = 1000
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
use_ttl = True
ttl_seconds = 30 

# Llamada a la función para insertar datos artificiales con TTL
insert_artificial_data(session, num_records, start_date, end_date, use_ttl, ttl_seconds)

Se han insertado 1000 registros artificiales en la tabla 'sales'.


In [12]:
def fetch_all_sales(session):
    select_all_query = "SELECT * FROM sales"
    rows = session.execute(select_all_query)
    
    sales_data = []
    for row in rows:
        sales_data.append({
            'order_id': row.order_id,
            'customer_id': row.customer_id,
            'product_id': row.product_id,
            'quantity': row.quantity,
            'order_date': row.order_date,
            'total_amount': row.total_amount
        })
    
    return sales_data

In [16]:
# Llamada a la función para obtener todo el contenido de la tabla sales
sales_data = fetch_all_sales(session)

# Mostrar los datos
for sale in sales_data:
    print(sale)

{'order_id': UUID('e1d8aa9f-09d6-4249-9672-cc35935ac209'), 'customer_id': UUID('5249309c-77c9-483f-a3e4-394b10f79369'), 'product_id': UUID('78dcef15-d44e-4c0d-861f-7d51eca7954f'), 'quantity': 4, 'order_date': datetime.datetime(2023, 12, 31, 7, 34, 20), 'total_amount': Decimal('327.88')}
{'order_id': UUID('d8173758-0dc0-48e1-af11-4bc229ba38db'), 'customer_id': UUID('8ef646d9-666e-44b8-b795-55d2648f1ed6'), 'product_id': UUID('82ee69e7-8f9a-41a9-ab20-ff839bac3dd7'), 'quantity': 6, 'order_date': datetime.datetime(2023, 3, 5, 18, 40, 43), 'total_amount': Decimal('494.68')}
{'order_id': UUID('780d43ee-84f8-4e4d-9c63-be9284f4eec1'), 'customer_id': UUID('6ef3c8a2-da7a-46f0-86e0-d81135f9ea53'), 'product_id': UUID('036c9413-1753-48a5-8242-1b6db01c6ea2'), 'quantity': 7, 'order_date': datetime.datetime(2023, 4, 8, 18, 2, 34), 'total_amount': Decimal('138.96')}
{'order_id': UUID('3f07c464-d9d6-4014-bcbf-7a7332178b2e'), 'customer_id': UUID('fc2ffdca-2de2-4f42-beb9-b814bbcb6895'), 'product_id': UUID(

In [17]:
import pandas as pd
from IPython.display import display

def display_documents_as_table(documents):
    """
    Muestra un array de documentos como una tabla en un Jupyter Notebook.
    :param documents: Lista de diccionarios donde cada diccionario representa un documento.
    """
    df = pd.DataFrame(documents)
    display(df)

In [18]:
# Llamada a la función para obtener todo el contenido de la tabla sales
sales_data = fetch_all_sales(session)
display_documents_as_table(sales_data)

,order_id,customer_id,product_id,quantity,order_date,total_amount
0,6cc3c07f-a626-4270-a357-ece2096dd257,caea7623-e6c7-4340-8230-44b32a8da7d8,1ed7ab07-3c32-4506-b02a-bd5fb362a246,3,2024-05-31 17:25:03.021,150.75


## Cerrar la Conexión

Finalmente, cerramos la conexión al clúster de Cassandra.

In [11]:
# Cerrar la conexión
cluster.shutdown()